In [1]:
import os
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = "true"

# import modules
from shop.models import *

In [15]:
total = Payment.objects.annotate(after_discount_price= F('price') - F('discount')).order_by('-after_discount_price')[:10]

for payment in total:
    print(
        f"ID: {payment.id}, PRICE: {payment.price}, "
        f"DISCOUNT: {payment.discount}, AFTER_DISCOUNT: {payment.after_discount_price}"
    )

ID: 92, PRICE: 1200500.00, DISCOUNT: 29433.25, AFTER_DISCOUNT: 1171066.75
ID: 82, PRICE: 1200280.00, DISCOUNT: 46229.40, AFTER_DISCOUNT: 1154050.60
ID: 137, PRICE: 1200690.00, DISCOUNT: 71407.25, AFTER_DISCOUNT: 1129282.75
ID: 105, PRICE: 1200390.00, DISCOUNT: 105019.11, AFTER_DISCOUNT: 1095370.89
ID: 45, PRICE: 1218900.00, DISCOUNT: 126859.95, AFTER_DISCOUNT: 1092040.05
ID: 7, PRICE: 1201200.00, DISCOUNT: 113446.20, AFTER_DISCOUNT: 1087753.80
ID: 18, PRICE: 1202190.00, DISCOUNT: 121922.64, AFTER_DISCOUNT: 1080267.36
ID: 77, PRICE: 379000.00, DISCOUNT: 19397.00, AFTER_DISCOUNT: 359603.00
ID: 127, PRICE: 320450.00, DISCOUNT: 14578.90, AFTER_DISCOUNT: 305871.10
ID: 125, PRICE: 320399.00, DISCOUNT: 17939.55, AFTER_DISCOUNT: 302459.45


In [18]:
total = Payment.objects.annotate(after_discount_price= F('price') - F('discount')
                                ).filter(after_discount_price__gt=500000).order_by('-after_discount_price')[:10]

for payment in total:
    print(
        f"ID: {payment.id}, PRICE: {payment.price}, "
        f"DISCOUNT: {payment.discount}, AFTER_DISCOUNT: {payment.after_discount_price}"
    )

ID: 92, PRICE: 1200500.00, DISCOUNT: 29433.25, AFTER_DISCOUNT: 1171066.75
ID: 82, PRICE: 1200280.00, DISCOUNT: 46229.40, AFTER_DISCOUNT: 1154050.60
ID: 137, PRICE: 1200690.00, DISCOUNT: 71407.25, AFTER_DISCOUNT: 1129282.75
ID: 105, PRICE: 1200390.00, DISCOUNT: 105019.11, AFTER_DISCOUNT: 1095370.89
ID: 45, PRICE: 1218900.00, DISCOUNT: 126859.95, AFTER_DISCOUNT: 1092040.05
ID: 7, PRICE: 1201200.00, DISCOUNT: 113446.20, AFTER_DISCOUNT: 1087753.80
ID: 18, PRICE: 1202190.00, DISCOUNT: 121922.64, AFTER_DISCOUNT: 1080267.36


In [74]:
customers = Customer.objects.annotate(full_name=Concat(F('first_name'), Value(' '), F('last_name'))
                                     ).order_by('full_name')[:5].values('id', 'email', 'address', 'full_name')

total = list(customers)


print(json.dumps(total, indent=4, sort_keys=False))

[
    {
        "id": 17,
        "email": "anantaya.deena@gmail.com",
        "address": {
            "district": "Yan Nawa",
            "location": "60 Thanon Chan Road",
            "province": "Bangkok",
            "postal_code": 10120
        },
        "full_name": "Anantaya Tontong"
    },
    {
        "id": 25,
        "email": "bancha.zeyou@gmail.com",
        "address": {
            "district": "Watthana",
            "location": "6 Thong Lo Road",
            "province": "Bangkok",
            "postal_code": 10110
        },
        "full_name": "Bancha Kittisompong"
    },
    {
        "id": 19,
        "email": "chayapol.231@gmail.com",
        "address": {
            "district": "Hang Chat",
            "location": "160 Lampang Road",
            "province": "Lampang",
            "postal_code": 52190
        },
        "full_name": "Chayapol Komprach"
    },
    {
        "id": 4,
        "email": "dejwit.tt@gmail.com",
        "address": {
            "district":

In [25]:
average_price = Product.objects.filter(remaining_amount__gt=0).aggregate(avg=Avg('price'))

print(average_price)

{'avg': Decimal('28593.848484848485')}


In [31]:
sum_price = CartItem.objects.filter(cart__create_date__month = 5). aggregate(sum=Sum('product__price'))

print(sum_price)

{'sum': Decimal('9912555.00')}


In [75]:
products_in_categories = Product.objects.filter(Q(categories__name='Electronics') | Q(categories__name='Jewelry'),
    price__gte=8000.00, price__lte=50000.00).values('categories__name').annotate(product_count=Count('id'))

for entry in products_in_categories:
    print(f"PRODUCT CATEGORY NAME: {entry['categories__name']}, PRODUCT COUNT: {entry['product_count']}")

PRODUCT CATEGORY NAME: Electronics, PRODUCT COUNT: 8
PRODUCT CATEGORY NAME: Jewelry, PRODUCT COUNT: 1


In [41]:
products = Product.objects.filter(categories__name='Information Technology').order_by('id').values('id', 'name', 'categories__name')

for product in products:
    print(f"PRODUCT ID: {product['id']}, PRODUCT NAME: {product['name']}, PRODUCT CATEGORY: {product['categories__name']}")

PRODUCT ID: 1, PRODUCT NAME: Smartphone, PRODUCT CATEGORY: Information Technology
PRODUCT ID: 2, PRODUCT NAME: Laptop, PRODUCT CATEGORY: Information Technology
PRODUCT ID: 3, PRODUCT NAME: Smart TV, PRODUCT CATEGORY: Information Technology
PRODUCT ID: 4, PRODUCT NAME: Bluetooth Earphones, PRODUCT CATEGORY: Information Technology
PRODUCT ID: 5, PRODUCT NAME: Tablet, PRODUCT CATEGORY: Information Technology
PRODUCT ID: 6, PRODUCT NAME: Gaming Console, PRODUCT CATEGORY: Information Technology
PRODUCT ID: 7, PRODUCT NAME: Digital Camera, PRODUCT CATEGORY: Information Technology
PRODUCT ID: 8, PRODUCT NAME: Wireless Router, PRODUCT CATEGORY: Information Technology
PRODUCT ID: 9, PRODUCT NAME: Portable Power Bank, PRODUCT CATEGORY: Information Technology
PRODUCT ID: 10, PRODUCT NAME: Smartwatch, PRODUCT CATEGORY: Information Technology
PRODUCT ID: 69, PRODUCT NAME: Notebook HP Pavilion Silver, PRODUCT CATEGORY: Information Technology
PRODUCT ID: 72, PRODUCT NAME: Notebook HP Pavilion Silver,

In [68]:
ProductCategory.objects.filter(name='Books').update(name = 'Books and Toys')

0

In [76]:
toys_and_games = ProductCategory.objects.get(name='Toys and Games')
books_and_toys = ProductCategory.objects.get(name='Books and Toys')

product_toys_and_games = Product.objects.filter(categories=toys_and_games)

for product in product_toys_and_games:
    product.categories.add(books_and_toys)
    product.categories.remove(toys_and_games) 

count = Product.objects.filter(categories=books_and_toys).count()

print(count)

# toys_and_games.delete()


DoesNotExist: ProductCategory matching query does not exist.